In [1]:
# !pip install tavily-python 
!pip install -U tavily-python langchain_community --quiet
!pip install autogen-agentchat~=0.2 autogen psutil --quiet
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai langchain-anthropic termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = "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"

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [4]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langgraph.graph import END, StateGraph, START

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.0-8b-instruct"
API_KEY= "alanliuxiang"

# INFERENCE_SERVER_URL = "https://api.feidaapi.com/"
# MODEL_NAME = "gpt-4o"
# API_KEY= "sk-TsoMJYTiW1Ya1AatboGdUGXEdCOAWeCeIbvUCkNMonrCHoQ7"

In [5]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.types import Command, interrupt
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
import operator

model = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# tools
tool = TavilySearchResults(max_results=4)

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    
class Agent:
    
    def __init__(self, model, tools, system=""):
        self.system = system
        
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], 
                                       name=t['name'], 
                                       content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!"""

# model = ChatOpenAI(model="gpt-4-turbo") #reduce inference cost
abot = Agent(model, [tool], system=prompt)

question = """Which country won the ICC t20  World Cup in 2024? 
What is the name of the capital city of that county?"""

messages = [HumanMessage(content=question)]
result = abot.graph.invoke({'messages': messages})
print(result['messages'][-1].content)

# """
# Calling: {'name': 'tavily_search_results_json', 
# 'args': {'query': 'ICC T20 World Cup 2024 winner'}, 'id': 'call_3X2XeLNJ3RSHZVxYgJyNGLlI', 'type': 'tool_call'}

# Back to the model!
# Calling: {'name': 'tavily_search_results_json', 
# 'args': {'query': 'capital city of India'}, 'id': 'call_vkgaIdrtoIecRVpODo7WyatY', 'type': 'tool_call'}

# Back to the model!
# India won the ICC T20 World Cup in 2024. 
# The capital city of India is New Delhi.
# """


I'm sorry for the confusion, but as of now, the ICC T20 World Cup in 2024 has not yet taken place. Therefore, I cannot provide the name of the winning country or its capital city. Please check back after the event for accurate information.I'm sorry for the confusion, but as of now, the ICC T20 World Cup in 2024 has not yet taken place. Therefore, I cannot provide the name of the winning country or its capital city. Please check back after the event for accurate information.
